In [1]:
# !git pull

In [2]:
import malaya_speech.train as train
import malaya_speech






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
from glob import glob
import os

files = glob('../speech-bahasa/LibriSpeech/*/*/*/*.trans.txt')
len(files)

672

In [4]:
audios, texts = [], []

for file in files:
    
    p = os.path.split(file)[0]
    with open(file) as fopen:
        d = list(filter(None, fopen.read().split('\n')))
    
    for row in d:
        splitted = row.split()
        s = os.path.join(p, splitted[0] + '.flac')
        text = ' '.join(splitted[1:])
        if not os.path.exists(s):
            print(s, 'not exist')
            continue
        
        audios.append(s)
        texts.append(text)

    
len(audios), len(texts)

(31159, 31159)

In [5]:
import unicodedata
import re


def preprocessing_text(string, reject = ',.!?#@\U0001f9da_，é)^🍪❤！' + '️'):
    for c in reject:
        string = string.replace(c, ' ')
        
    string = unicodedata.normalize('NFC', string.lower())
    return re.sub(r'[ ]+', ' ', string).strip()

In [6]:
cleaned_texts = [preprocessing_text(t) for t in texts]

In [7]:
unique_chars = malaya_speech.char.generate_vocab(cleaned_texts)
unique_chars

['<PAD>',
 '<EOS>',
 ' ',
 'e',
 't',
 'a',
 'o',
 'n',
 'i',
 'h',
 's',
 'r',
 'd',
 'l',
 'u',
 'm',
 'c',
 'w',
 'f',
 'g',
 'y',
 'p',
 'b',
 'v',
 'k',
 "'",
 'x',
 'j',
 'q',
 'z']

In [8]:
import json

with open('librispeech-sst-vocab.json', 'w') as fopen:
    json.dump(unique_chars, fopen)

In [9]:
from tqdm import tqdm

def generator(maxlen = 16):
    for i in tqdm(range(len(audios))):
        try:
            if '.mp3' in audios[i]:
                wav_data, sr = mp3_to_wav(audios[i])
            else:
                wav_data, sr = malaya_speech.load(audios[i])
                
            if (len(wav_data) / sr) > maxlen or len(cleaned_texts[i]) < 5:
                print(f'skipped {audios[i]}')
                continue

            yield {
                'waveforms': wav_data.tolist(),
                'waveform_lens': [len(wav_data)],
                'targets': malaya_speech.char.encode(cleaned_texts[i], add_eos = False,
                                                     lookup = unique_chars),
                'raw_transcript': [cleaned_texts[i]],
            }
        except Exception as e:
            print(e)
            
generator = generator()

In [10]:
import os
import tensorflow as tf

os.system('rm training-librispeech/data/*')
DATA_DIR = os.path.expanduser('training-librispeech/data')
tf.gfile.MakeDirs(DATA_DIR)

In [11]:
shards = [{'split': 'train', 'shards': 999}, {'split': 'dev', 'shards': 1}]

In [12]:
train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'librispeech')

  0%|          | 0/31159 [00:00<?, ?it/s]


INFO:tensorflow:Generating case 0.


  0%|          | 3/31159 [00:00<20:20, 25.52it/s]

skipped ../speech-bahasa/LibriSpeech/train-clean-100/3664/178355/3664-178355-0001.flac


  0%|          | 16/31159 [00:00<22:04, 23.50it/s]

...

100%|█████████▉| 31155/31159 [19:51<00:00, 35.38it/s]

skipped ../speech-bahasa/LibriSpeech/test-clean/8224/274381/8224-274381-0004.flac
skipped ../speech-bahasa/LibriSpeech/test-clean/8224/274381/8224-274381-0005.flac
skipped ../speech-bahasa/LibriSpeech/test-clean/8224/274381/8224-274381-0006.flac
skipped ../speech-bahasa/LibriSpeech/test-clean/8224/274381/8224-274381-0008.flac
skipped ../speech-bahasa/LibriSpeech/test-clean/8224/274381/8224-274381-0009.flac
skipped ../speech-bahasa/LibriSpeech/test-clean/8224/274381/8224-274381-0012.flac
skipped ../speech-bahasa/LibriSpeech/test-clean/8224/274381/8224-274381-0014.flac


100%|██████████| 31159/31159 [19:51<00:00, 26.15it/s]

skipped ../speech-bahasa/LibriSpeech/test-clean/8224/274381/8224-274381-0016.flac
skipped ../speech-bahasa/LibriSpeech/test-clean/8224/274381/8224-274381-0017.flac



INFO:tensorflow:Generated 29175 Examples
INFO:tensorflow:Shuffling data...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`

INFO:tensorflow:Data shuffled.
